In [1]:
import os
import json

from react_agent.src.config.system_parameters import CodebaseSearcherSettings

from react_agent.src.agent_tools.source_code_retriever import SourceCodeRetriever

from react_agent.src.util.abap_repository import ABAPClassRepository
from react_agent.src.util.memory_manager import MemoryManager
from react_agent.src.util.code_summarizer import CodeSummarizer

In [2]:
notebook_dir = os.path.abspath("")

file_path = os.path.abspath(
    os.path.join(notebook_dir, "./", "resources", "classes_to_index.json")
)

with open(file_path, encoding="utf8") as f:
    json_file = json.load(f)

In [3]:
classes: list[tuple] = []

for my_class in json_file["classes"]:
    classes.append((my_class["name"].lower(), my_class["description"]))

for my_class in classes:
    print(f"Class name: {my_class[0]}, Description: {my_class[1]}")

Class name: cl_edoc_map_aif, Description: eDocument Mapping AIF
Class name: cl_edoc_map_ubl, Description: eDocument Mapping UBL
Class name: cl_edoc_map_ubl_ca, Description: eDocument Mapping UBL - FICA
Class name: cl_edoc_map_ubl_ci, Description: eDocument Mapping UBL - CI
Class name: cl_edoc_map_ubl_dbna, Description: eDocument DBNA Mapping Class
Class name: cl_edoc_map_ubl_fi, Description: eDocument Mapping UBL FI
Class name: cl_edoc_map_ubl_fr, Description: eDocument EU France Mapping Class
Class name: cl_edocument_fr_inv, Description: 
Class name: cl_edoc_map_ubl_file, Description: eDocument Mapping UBL: External Files
Class name: cl_edoc_map_ubl_liv, Description: eDocument UBL: Logistics Invoice Verification
Class name: cl_edoc_map_ubl_ord, Description: eDocument Mapping UBL
Class name: cl_edoc_map_ubl_sd, Description: eDocument Mapping UBL SD
Class name: cl_edoc_map_ubl_stlmt, Description: UBL mapping for Settlement Document
Class name: cl_reex_edoc_map_ubl_eu, Description: eDocu

In [4]:
CODE_SEARCHER_SETTINGS = CodebaseSearcherSettings()

MEMORY_MANAGER = MemoryManager(
    memory_store_type="Postgres",
    namespace=CODE_SEARCHER_SETTINGS.namespace,
    embedding_fields=["description"],
)

In [5]:
code_lookup = SourceCodeRetriever()

for my_class in classes:
    print("Class name: ", my_class[0])
    class_name = my_class[0]
    class_description = my_class[1]

    class_code = code_lookup._run(class_name=class_name)

    print("\t- Started indexing")
    repository = ABAPClassRepository(source_code=class_code)
    print(f"\t- Finished indexing, number of methods: {len(repository.classes[class_name].keys())}")

    for method_name in repository.classes[class_name].keys():
        if not MEMORY_MANAGER.is_memory_present(f"{class_name}.{method_name}"):
            print(f"\t- Summarizing and adding method: {method_name}")
            # Summarize the code of the method and add it to the memory
            memory_content = {
                "description": CodeSummarizer.summarize_code(
                    f"Class description: {class_description}\n"
                    + repository.get_content_by_class_and_method(
                        class_name=class_name, method_name=method_name
                    )
                ),
                "code": repository.get_content_by_class_and_method(
                    class_name=class_name, method_name=method_name
                ),
            }
            MEMORY_MANAGER.add_memory(
                memory_title=f"{class_name}.{method_name}",
                memory_content=memory_content,
            )
        else:
            print(f"\t- Skipping method: {method_name}")
            continue

Class name:  cl_edoc_map_aif
	- Started indexing
	- Finished indexing, number of methods: 17
	- Skipping method: add_error_message
	- Skipping method: constructor
	- Skipping method: convert_error_message
	- Skipping method: determine_mapping_method
	- Skipping method: determine_method_param_value
	- Skipping method: format_amount
	- Skipping method: format_negative_amount
	- Skipping method: get_fix_value
	- Skipping method: get_value_mapping
	- Skipping method: init_fix_values
	- Skipping method: init_mapping
	- Skipping method: init_value_mapping
	- Skipping method: map_interface
	- Skipping method: pack_interface_id
	- Skipping method: raise_exception
	- Skipping method: unpack_interface_id
	- Skipping method: get_multiple_value_mapping
Class name:  cl_edoc_map_ubl
	- Started indexing
	- Finished indexing, number of methods: 172
	- Skipping method: init_header_item_texts
	- Skipping method: get_sepa_mandate
	- Skipping method: get_party_with_param_type
	- Skipping method: get_party